In [2]:
%load_ext autoreload
%autoreload 2
from collections import OrderedDict
from datasets import load_dataset
from more_itertools import chunked
from itda import ITDAConfig, ITDA
from tqdm.auto import tqdm
import numpy, torch
import json, os
os.makedirs("dataset", exist_ok=True)

In [3]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16, device_map="balanced")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
torch.set_grad_enabled(False)

In [5]:
import numpy as np
nf4 = np.asarray(
    [
        -1.0,
        -0.6961928009986877,
        -0.5250730514526367,
        -0.39491748809814453,
        -0.28444138169288635,
        -0.18477343022823334,
        -0.09105003625154495,
        0.0,
        0.07958029955625534,
        0.16093020141124725,
        0.24611230194568634,
        0.33791524171829224,
        0.44070982933044434,
        0.5626170039176941,
        0.7229568362236023,
        1.0,
    ]
)
image_max = 6.0

In [6]:
prompts_dataset = load_dataset("opendiffusionai/cc12m-cleaned")
prompts_iterator = prompts_dataset["train"]["caption_llava_short"]
guidance_scale = 3.5
num_inference_steps = 1
batch_size = 32
width = 256
height = 256
d_model = 3072
itda_config = ITDAConfig(
    d_model=d_model,
    target_l0=64,
    loss_threshold=0.6,
    add_error=True,
    fvu_loss=True,
    subtract_mean=False,
)

In [7]:
len(prompts_iterator)

8531902

In [8]:
normalize = True
device = torch.device("cuda:0")
itda = ITDA(itda_config, dtype=torch.bfloat16, device=device)

In [18]:
itda = itda.load_from_disk("itda_72k_steps_256").to(device)
stds = itda.mean_x

In [19]:
from caching_utils import save_itda_outputs
from scored_storage import ScoredStorage
from pathlib import Path

feature_acts = ScoredStorage(
    Path("itda_data") / "feature_acts.db",
    3, 64
)

In [ ]:
for i, prompts in enumerate(chunked((bar := tqdm(prompts_iterator[80000:100000])), batch_size)):
    with torch.inference_mode():
        for m in pipe.transformer.modules():
            m._forward_hooks = OrderedDict()
        text_outputs = {}
        image_outputs = {}
        timestep = 0
        def save_hook(self, input, output):
            text_outputs[timestep] = output[0]
            image_outputs[timestep] = output[1]
        pipe.transformer.transformer_blocks[18].register_forward_hook(save_hook)
        height = 256
        width = 256
        def callback_on_step_end(self, i, t, kwargs):
            global timestep
            timestep = i
            return {}
        pipe.set_progress_bar_config(disable=True)
        with torch.inference_mode():
            latents = pipe(
                prompts,
                height=height,
                width=width,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                max_sequence_length=512,
                generator=torch.Generator("cpu").manual_seed(0),
                return_dict=False,
                callback_on_step_end=callback_on_step_end,
                output_type="latent",
            )[0]
        latents_reshaped = pipe._unpack_latents(latents, height, width, pipe.vae_scale_factor)
        latents_to_be_compressed = latents_reshaped.cpu().float().numpy()
        latents_to_save = (latents_to_be_compressed / image_max).clip(-1, 1)
        latents_to_save = np.abs(latents_to_save[..., None] - nf4).argmin(-1).astype(np.uint8)
        latents_to_save = (
            (latents_to_save[..., ::2] & 0x0F)
            | ((latents_to_save[..., 1::2] << 4) & 0xF0))
        
        x = image_outputs[0].to(torch.bfloat16).to(device)
        x = x.reshape(-1, d_model)

        if normalize:
            if stds is None:
                stds = x.std(dim=0, unbiased=False)
            x = x / stds

        out = itda(x,x)

        weights = out.weights.float().cpu().numpy()
        indices = out.indices.float().cpu().numpy()

        weights = weights.reshape(image_outputs[0].shape[0], image_outputs[0].shape[1], -1)
        indices = indices.reshape(image_outputs[0].shape[0], image_outputs[0].shape[1], -1)

        save_itda_outputs(
            weights, indices, prompts=prompts, images=latents_to_be_compressed, feature_acts=feature_acts, save_image_activations=True, step=i
        )

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 